<a href="https://colab.research.google.com/github/sirTucano/asd/blob/master/Freight_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import libraries

In [ ]:
import pandas as pd
import os
import numpy as np

total Cost Center data for Outward Freight

In [ ]:
list_CCO = ['Fiscal Year','Period','Cost element name','Cost element descr.','Vendor','Vendor Name','Document Date','Posting Date','Value in Obj. Crcy','Object Currency','Ref. document number','Document Header Text','Plant', 'Purchase order text']
list_CCI = ['Fiscal Year','Period','Cost element name','Cost element descr.','Vendor','Vendor Name','Document Date','Posting Date','Value in Obj. Crcy','Object Currency','Ref. document number','Document Header Text','Plant','Purchasing Document','Purchase order text']
list_PO = ['Cal. year / month', 'Purchase order number', 'Plant', 'Material','Unnamed: 4','Mode of Transport','Global Vendor','Country','Quantity of GRs']
list_Shipment = ['Shipment number', 'Plant.', 'Forwarding Agent', 'Unnamed: 3', 'Incoterms','Incoterms 2', 'Mode of Transport', 'Cal. year / month', 'Act.goods issue date', 'Qty in KG', '# OD Item']

path_CCO = '/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/CC_Outbound/'
path_CCI = '/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/CC_Inbound/'
path_PO = '/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/PO/'
path_Shipment = '/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/Shipment/'
path_Append = '/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/Append/'

sheetname_CCO = 'Sheet1'
sheetname_CCI = 'Sheet1'
sheetname_PO = 'Sheet1'
sheetname_shipment = 'Outbound_Deliveries'

declare merger function

In [ ]:
def merger(path_in,path_out,output,column_names,sheetname,as_type):
  global df
  df = pd.DataFrame()
  for filename in os.listdir(path_in):
    temp_df = pd.read_excel("{}/{}".format(path_in,filename),sheet_name = sheetname,dtype={as_type:str})
    temp_df = temp_df[column_names]
    df = df.append(temp_df,sort='True')
  df = df.to_csv(path_out+output)

run merger function for all datasets
(no data reduction at this point)

In [ ]:
merger(path_CCO,path_Append,'total_CCO.csv',list_CCO,sheetname_CCO,as_type=False)
merger(path_CCI,path_Append,'total_CCI.csv',list_CCI,sheetname_CCI,as_type=False)
merger(path_Shipment,path_Append,'total_Shipment.csv',list_Shipment,sheetname_shipment,'Cal. year / month')
merger(path_PO,path_Append,'total_PO.csv',list_PO,sheetname_PO,'Cal. year / month')

collect merged datasets

In [ ]:
df_CCO = pd.read_csv(path_Append+'total_CCO.csv')
df_CCI = pd.read_csv(path_Append+'total_CCI.csv')
df_Shipment = pd.read_csv(path_Append+'total_Shipment.csv',dtype={'Cal. year / month':str})
df_PO = pd.read_csv(path_Append+'total_PO.csv',dtype={'Cal. year / month':str})

print('Total PO+STO items:\n',df_PO['Purchase order number'].count())

#reduce STOs from PO db:
# df_PO['PO/STO'] = np.where(df_PO['Purchase order number'].str.len()<9,'STO','PO')
# df_PO = df_PO[df_PO['PO/STO']=='PO']

print('Total PO items:\n',df_PO['Purchase order number'].count())
print('Total PO unique items:\n',df_PO['Purchase order number'].nunique(),'\n')

print('Total Inbound Invoice items:\n',df_CCI['Cost element descr.'].count())
print('Unique Inbound Invoice items:\n',df_CCI['Document Header Text'].nunique(),'\n')

print('Total Outbound Invoice items:\n',df_CCO['Cost element descr.'].count())
print('Unique Outbound Invoice items:\n',df_CCO['Document Header Text'].nunique(),'\n')

Total PO+STO items:
 32903
Total PO items:
 32903
Total PO unique items:
 32313 

Total Inbound Invoice items:
 31416
Unique Inbound Invoice items:
 11368 

Total Outbound Invoice items:
 231225
Unique Outbound Invoice items:
 146267 



In [ ]:
df_Shipment['Cal. year / month'].unique()

array(['11.2019', '10.2019', '12.2019', '08.2019', '07.2019', '09.2019',
       '05.2019', '04.2019', '06.2019', '01.2019', '02.2019', '03.2019',
       '01.2020', '02.2020', '03.2020', '04.2020', '05.2020', '06.2020',
       '08.2020', '07.2020', '09.2020'], dtype=object)

In [ ]:
df_Shipment.describe()

,Unnamed: 0,# OD Item,Qty in KG,Shipment number
count,153502.000000,153502.000000,153502.000000,1.535020e+05
mean,11039.843520,3.354549,1232.503515,4.120817e+06
std,6454.993906,3.530750,3201.829623,2.511151e+05
min,0.000000,1.000000,0.002000,3.591114e+06
25%,5482.000000,2.000000,50.000000,3.902192e+06
50%,10964.000000,2.000000,200.000000,4.115256e+06
75%,16446.000000,4.000000,944.575000,4.331012e+06
max,24082.000000,86.000000,130720.000000,4.574413e+06


In [ ]:
df_Shipment['Shipment number']

0         4034332
1         4041024
2         4041126
3         4041220
4         4041935
           ...   
153497    4574123
153498    4574129
153499    4574138
153500    4574180
153501    4574413
Name: Shipment number, Length: 153502, dtype: int64

**Print number of incorrect 'Document Header Text' in Outbound data:**

In [ ]:
df_manual_CCO = df_CCO
df_manual_CCO = df_manual_CCO.dropna(subset=['Document Header Text'])
df_manual_CCO = df_manual_CCO['Document Header Text'].apply(pd.to_numeric, errors='coerce')
print('Total number of incorrect Document Header Text:\n',df_manual_CCO.isna().sum())

Total number of incorrect Document Header Text:
 23064


Convert Inbound Document Header Text to string and take first 10 characters:

In [ ]:
df_CCI['Document Header Text'] = df_CCI['Document Header Text'].astype(str).str[:10]

print number of incorrect 'Document Header Text' in Inbound data:

In [ ]:
df_manual_CCI = df_CCI
df_manual_CCI = df_manual_CCI.dropna(subset=['Document Header Text'])
df_manual_CCI = df_manual_CCI['Document Header Text'].apply(pd.to_numeric, errors='coerce')
print('Total number of incorrect Document Header Text:\n',df_manual_CCI.isna().sum())

Total number of incorrect Document Header Text:
 17266


Drop all non-numeric values from the datasets and convert them to integer:

In [ ]:
df_CCO = (df_CCO.drop('Document Header Text', axis=1)
         .join(df_CCO['Document Header Text'].apply(pd.to_numeric, errors='coerce')))
df_CCO = df_CCO.dropna(subset=['Document Header Text'])

df_CCI = (df_CCI.drop('Document Header Text', axis=1)
         .join(df_CCI['Document Header Text'].apply(pd.to_numeric, errors='coerce')))
df_CCI = df_CCI.dropna(subset=['Document Header Text'])

df_PO = (df_PO.drop('Purchase order number', axis=1)
         .join(df_PO['Purchase order number'].apply(pd.to_numeric, errors='coerce')))
df_PO = df_PO.dropna(subset=['Purchase order number'])

df_CCO['Document Header Text'] = df_CCO['Document Header Text'].astype(int)
df_CCI['Document Header Text'] = df_CCI['Document Header Text'].astype(int)

Merge the datasets together to Inbound and Outbound:

In [ ]:
df_Outbound = pd.merge(df_CCO,df_Shipment,left_on='Document Header Text',right_on= 'Shipment number',how ='left')
df_Inbound = pd.merge(df_CCI,df_PO,left_on='Document Header Text',right_on= 'Purchase order number',how ='left')

In [ ]:
df_Outbound_2 = pd.merge(df_Shipment,df_CCO,left_on='Shipment number',right_on= 'Document Header Text',how ='left')

In [ ]:
df_Outbound_2['Cal. year / month'] = df_Outbound['Cal. year / month'].str.replace('.','/',regex=True)
df_Outbound_2.to_csv('/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/Outbound_2.csv')

In [ ]:
df_PO_invoice = pd.merge(df_PO,df_CCI,left_on='Purchase order number',right_on= 'Document Header Text',how ='left')

In [ ]:
df_Outbound['Cal. year / month'] = df_Outbound['Cal. year / month'].str.replace('.','/',regex=True)
df_Outbound['Shipment number'] = df_Outbound['Shipment number'].astype(str)
df_Outbound['Vendor'] = df_Outbound['Vendor'].astype(str)

df_Outbound_2['Cal. year / month'] = df_Outbound['Cal. year / month'].str.replace('.','/',regex=True)
df_Outbound_2['Shipment number'] = df_Outbound['Shipment number'].astype(str)
df_Outbound_2['Vendor'] = df_Outbound['Vendor'].astype(str)

Output them to csv:

In [ ]:
df_Outbound.to_csv('/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/Outbound.csv')

df_Inbound.to_csv('/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/Inbound.csv')
df_PO_invoice.to_csv('/content/drive/My Drive/Python/p_Freight_Analysis/Source Files/PO_invoice.csv')

In [ ]:
print('Total Outbound Invoice items:\n',df_Outbound['Document Header Text'].count())
print('Unique Outbound Invoice items:\n',df_Outbound['Document Header Text'].nunique())
print('Total Outbound Shipments items:\n',df_Outbound['Shipment number'].count())
print('Unique Outbound Shipments items:\n',df_Outbound['Shipment number'].nunique())

df_MA01_Inbound = df_Inbound[df_Inbound['Plant_y']=='MA01']
print('2601225005 Total Inbound items:\n','6506')
print('MA01 Total Inbound items:\n','6113')
print('MA01 Inbound items:\n',df_MA01_Inbound['Document Header Text'].count())

df_MA01_Outbound = df_Outbound[df_Outbound['Plant.']=='MA01']
print('MA01 Outbound items:\n',df_MA01_Outbound['Document Header Text'].count())

Total Outbound Invoice items:
 205898
Unique Outbound Invoice items:
 125805
Total Outbound Shipments items:
 205898
Unique Outbound Shipments items:
 114930
2601225005 Total Inbound items:
 6506
MA01 Total Inbound items:
 6113
MA01 Inbound items:
 2646
MA01 Outbound items:
 51207


In [ ]:
print('Total Inbound Invoice items:\n',df_Inbound['Document Header Text'].count())
print('Unique Inbound Invoice items:\n',df_Inbound['Document Header Text'].nunique())
print('Total Inbound PO items:\n',df_Inbound['Purchase order number'].count())
print('Unique Inbound PO items:\n',df_Inbound['Purchase order number'].nunique())

Total Inbound Invoice items:
 14606
Unique Inbound Invoice items:
 10948
Total Inbound PO items:
 8284
Unique Inbound PO items:
 6113
